# Import

In [1]:
# De-noising input data and applying auto-decoder

import tensorflow as tf
import argparse

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

C:\Users\Ali\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


# Parameters

In [2]:
#input size
input_size = 784
#re-constructed size
output_size = 784

# -------------------------------------------
# Following Hinton-Salakhutdinov Architecture

# 3 hidden layers for encoder
n_encoder_h_1 = 1000
n_encoder_h_2 = 500
n_encoder_h_3 = 250

# 3 hidden layers for decoder
n_decoder_h_1 = 250
n_decoder_h_2 = 500
n_decoder_h_3 = 1000


# Parameters
learning_rate = 0.01

training_epochs = 200
batch_size = 200
display_step = 1

# Batch Normalization

In [3]:
def layer_batch_normalization(x, n_out, phase_train):
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - n_out: integer, depth of input maps - number of sample in the batch 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - batch-normalized maps   
    """

    beta_init = tf.constant_initializer(value=0.0, dtype=tf.float32)
    beta = tf.get_variable("beta", [n_out], initializer=beta_init)
    
    gamma_init = tf.constant_initializer(value=1.0, dtype=tf.float32)
    gamma = tf.get_variable("gamma", [n_out], initializer=gamma_init)

    #tf.nn.moment: https://www.tensorflow.org/api_docs/python/tf/nn/moments
    #calculate mean and variance of x
    batch_mean, batch_var = tf.nn.moments(x, [0], name='moments')
    
    #tf.train.ExponentialMovingAverage:
    #https://www.tensorflow.org/api_docs/python/tf/train/ExponentialMovingAverage
    #Maintains moving averages of variables by employing an exponential decay.
    ema = tf.train.ExponentialMovingAverage(decay=0.9)
    ema_apply_op = ema.apply([batch_mean, batch_var])
    ema_mean, ema_var = ema.average(batch_mean), ema.average(batch_var)
    
    def mean_var_with_update():
        with tf.control_dependencies([ema_apply_op]):
            return tf.identity(batch_mean), tf.identity(batch_var)
        
    #tf.cond: https://www.tensorflow.org/api_docs/python/tf/cond
    #Return true_fn() if the predicate pred is true else false_fn()
    mean, var = tf.cond(phase_train, mean_var_with_update, lambda: (ema_mean, ema_var))

    reshaped_x = tf.reshape(x, [-1, 1, 1, n_out])
    normed = tf.nn.batch_norm_with_global_normalization(reshaped_x, mean, var, beta, gamma, 1e-3, True)
    #normed = tf.nn.batch_normalization(reshaped_x, mean, var, beta, gamma, 1e-3, True)
    
    return tf.reshape(normed, [-1, n_out])

# Definition of the Layer

In [4]:
def layer(x, weight_shape, bias_shape, phase_train):
    """
    Defines the network layers
    Using sigmoid to avoid sharp transitions in neurons
    input:
        - x: input vector of the layer
        - weight_shape: shape the the weight maxtrix
        - bias_shape: shape of the bias vector
        - phase_train: boolean tf.Variable, true indicates training phase
    output:
        - output vector of the layer after the matrix multiplication and non linear transformation
    """
    #initialize weights
    weight_init = tf.random_normal_initializer(stddev=(1.0/weight_shape[0])**0.5)
    W = tf.get_variable("W", weight_shape, initializer=weight_init)
    
    bias_init = tf.constant_initializer(value=0)
    b = tf.get_variable("b", bias_shape, initializer=bias_init)
    
    logits = tf.matmul(x, W) + b
    
    phase_train
    
    #apply the non-linear function after the batch normalization
    
    #return tf.nn.tanh(layer_batch_normalization(logits, weight_shape[1], phase_train))
    #return tf.nn.relu(layer_batch_normalization(logits, weight_shape[1], phase_train))
    return tf.nn.sigmoid(layer_batch_normalization(logits, weight_shape[1], phase_train))


# Definition of the Encoder Part

In [5]:
def encoder(x, n_code, phase_train):
    
    """
    Defines the network encoder part
    input:
        - x: input vector of the encoder
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder)
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector: reduced dimension
    """
    
    phase_train
    
    with tf.variable_scope("encoder"):
        
        with tf.variable_scope("h_1"):
            h_1 = layer(x, [input_size, n_encoder_h_1], [n_encoder_h_1], phase_train)

        with tf.variable_scope("h_2"):
            h_2 = layer(h_1, [n_encoder_h_1, n_encoder_h_2], [n_encoder_h_2], phase_train)

        with tf.variable_scope("h_3"):
            h_3 = layer(h_2, [n_encoder_h_2, n_encoder_h_3], [n_encoder_h_3], phase_train)

        with tf.variable_scope("code"):
            output = layer(h_3, [n_encoder_h_3, n_code], [n_code], phase_train)

    return output

# Definition of the Decoder Part

In [6]:
def decoder(x, n_code, phase_train):
    """
    Defines the network encoder part
    this mask is a noise added to the image 
    input:
        - x: input vector of the decoder - reduced dimension vector
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder) 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector: reconstructed dimension of the initial vector
    """
    
    phase_train
    
    with tf.variable_scope("decoder"):
        
        with tf.variable_scope("h_1"):
            h_1 = layer(x, [n_code, n_decoder_h_1], [n_decoder_h_1], phase_train)

        with tf.variable_scope("h_2"):
            h_2 = layer(h_1, [n_decoder_h_1, n_decoder_h_2], [n_decoder_h_2], phase_train)

        with tf.variable_scope("h_3"):
            h_3 = layer(h_2, [n_decoder_h_2, n_decoder_h_3], [n_decoder_h_3], phase_train)

        with tf.variable_scope("output"):
            output = layer(h_3, [n_decoder_h_3, output_size], [output_size], phase_train)

    return output

# Definition of the corrupt functions

In [7]:
def corrupt_1(x):
    
    #Take an input tensor and add uniform masking
    #Parameters
    #----------
    #x : Tensor/Placeholder
    #    Input to corrupt.
    #Returns
    #-------
    #x_corrupted : Tensor
    #    50 pct of values corrupted.

    return tf.multiply(x, tf.cast(tf.random_uniform(shape=tf.shape(x), minval=0, maxval=2, dtype=tf.int32), tf.float32))


def corrupt_2(x):
    """
    Take an input tensor and add uniform masking.
    input:
        - x : input vector - to be corrupted
    output:
        - output vector : half of the values are corrupted
    """
    
    #https://www.tensorflow.org/api_docs/python/tf/random_uniform
    #Outputs random values from a uniform distribution.
    #The lower bound minval is included in the range, while the upper bound maxval is excluded.
    #here 0 or 1
    c = tf.random_uniform(shape=tf.shape(x),  minval=0, maxval=2, dtype=tf.int32)
    
    #https://www.tensorflow.org/api_docs/python/tf/cast
    #change the type to float
        
    return x * tf.cast(c, tf.float32)

# Definition of the Loss

In [8]:
def loss(output, x):
    """
    Compute the loss of the auto-encoder
    intput:
        - output: the output of the decoder
        - x: true value of the sample batch - this is the input of the encoder
        
        the two have the same shape (batch_size * num_of_classes)
    output:
        - loss: loss of the corresponding batch (scalar tensor)
    
    """
    with tf.variable_scope("training"):
        
        l2_measure = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x)), 1))
        train_loss = tf.reduce_mean(l2_measure)
        train_summary_op = tf.summary.scalar("train_cost", train_loss)
        return train_loss, train_summary_op

# Training Function

In [9]:
def training(cost, global_step):
    """
    defines the necessary elements to train the network
    
    intput:
        - cost: the cost is the loss of the corresponding batch
        - global_step: number of batch seen so far, it is incremented by one 
        each time the .minimize() function is called
    """
    # Using Adam for optimization
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-08, use_locking=False, name='Adam')
    train_op = optimizer.minimize(cost, global_step=global_step)
    return train_op

# Evaluation Function

In [10]:
def evaluate(output, x, x_tilde):
    """
    evaluates the accuracy on the validation set 
    input:
        -output: prediction vector of the network for the validation set
        -x: true value for the validation set
        -x_tilde: corrupted image with a level c
    output:
        - val_loss: loss of the autoencoder
        - in_image_op: input image 
        - out_image_op:reconstructed image 
        - val_summary_op: summary of the loss
    """
    
    with tf.variable_scope("validation"):
        
        #input of the autoencoder is the corrupted image
        in_image_op = image_summary("input_image", x_tilde)
        
        #output image
        out_image_op = image_summary("output_image", output)
        
        #the validation loss is computed using the original image 
        l2_norm = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x, name="val_diff")), 1))
        
        val_loss = tf.reduce_mean(l2_norm)
        
        val_summary_op = tf.summary.scalar("val_cost", val_loss)
        
        return val_loss, in_image_op, out_image_op, val_summary_op

# Image Summary

In [11]:
def image_summary(label, tensor):
    #tf.summary.image: https://www.tensorflow.org/api_docs/python/tf/summary/image
    #Outputs a Summary protocol buffer with images.
    
    tensor_reshaped = tf.reshape(tensor, [-1, 28, 28, 1])
    return tf.summary.image(label, tensor_reshaped)

# Main Function

In [13]:
if __name__ == '__main__':

    # passing arguments through command line

    # if a python file, please use the 4 lines bellow and comment the "n_code = '1'"
    #parser = argparse.ArgumentParser(description='Autoencoder')
    #parser.add_argument('n_code', nargs=1, type=str)
    #args = parser.parse_args()
    #n_code = args.n_code[0]

    # if a jupyter file, please comment the 4 above and use the one bellow
    n_code = '2'

    model_path = 'C:/Users/Ali/logs/restore_2/model.ckpt'
    #model_path = 'C:/Users/Ali/logs/restore/'

    with tf.Graph().as_default():

        with tf.variable_scope("autoencoder_model"):

            # the input variables are first define as placeholder
            # a placeholder is a variable/data which will be assigned later
            # image vector & label, phase_train is a boolean
            # MNIST data image of shape 28*28=784
            x = tf.placeholder("float", [None, 784])

            phase_train = tf.placeholder(tf.bool)

            # the parameter c is also defined as a placeholder
            c = tf.placeholder(tf.float32)

            #x_tilde = x*(1.0 - c) + corrupt_x(x)*c
            x_tilde = corrupt_1(x)

            # define the encoder
            code = encoder(x_tilde, int(n_code), phase_train)

            # define the decoder
            output = decoder(code, int(n_code), phase_train)

            # compute the loss
            cost, train_summary_op = loss(output, x)

            # initialize the value of the global_step variable
            # recall: it is incremented by one each time the .minimise() is called
            global_step = tf.Variable(0, name='global_step', trainable=False)

            train_op = training(cost, global_step)

            # evaluate the accuracy of the network (done on a validation set)
            eval_op, in_image_op, out_image_op, val_summary_op = evaluate(output, x, x_tilde)

            summary_op = tf.summary.merge_all()

            # save and restore variables to and from checkpoints.
            saver = tf.train.Saver()

            # defines a session
            # A session allows to execute graphs or part of graphs
            # It allocates resources (on one or more machines) for that
            # and holds the actual values of intermediate results and variables
            sess = tf.Session()

            # summary writer
            # https://www.tensorflow.org/api_docs/python/tf/summary/FileWriter
            train_writer = tf.summary.FileWriter(model_path, graph=sess.graph)
            val_writer = tf.summary.FileWriter(model_path, graph=sess.graph)

            # initialization of the variables
            init_op = tf.global_variables_initializer()

            sess.run(init_op)

            # Training cycle
            for epoch in range(training_epochs):

                avg_cost = 0.
                total_batch = int(mnist.train.num_examples/batch_size)

                # Loop over all batches
                for i in range(total_batch):

                    minibatch_x, minibatch_y = mnist.train.next_batch(batch_size)

                    # Fit training using batch data
                    # the training is done using the training dataset
                    _, new_cost, train_summary = sess.run([train_op, cost, train_summary_op], feed_dict={
                                                          x: minibatch_x, phase_train: True, c: 1.0})

                    train_writer.add_summary(train_summary, sess.run(global_step))

                    # Compute average loss
                    avg_cost += new_cost/total_batch

                # Display logs per epoch step
                if epoch % display_step == 0:

                    print("Epoch:", '%04d' % (epoch+1),
                          "cost =", "{:.9f}".format(avg_cost))

                    train_writer.add_summary(
                        train_summary, sess.run(global_step))

                    validation_loss, in_image, out_image, val_summary = sess.run([eval_op, in_image_op, out_image_op, val_summary_op], feed_dict={
                                                                                 x: mnist.validation.images, phase_train: False, c: 1.0})

                    val_writer.add_summary(in_image, sess.run(global_step))
                    val_writer.add_summary(out_image, sess.run(global_step))
                    val_writer.add_summary(val_summary, sess.run(global_step))

                    print("Validation Loss:", validation_loss)

                    save_path = saver.save(sess, model_path)
                    print("Model saved in file: %s" % save_path)

            print("Optimization Done")

            test_loss = sess.run(eval_op, feed_dict={
                                 x: mnist.test.images, phase_train: False, c: 0.0})

            print("Test Loss:", test_loss)

Epoch: 0001 cost = 12.328632899
Validation Loss: 11.1034
Model saved in file: C:/Users/Ali/logs/restore_2/model.ckpt
Epoch: 0002 cost = 10.741893730
Validation Loss: 9.96536
Model saved in file: C:/Users/Ali/logs/restore_2/model.ckpt
Epoch: 0003 cost = 9.664100304
Validation Loss: 9.113872
Model saved in file: C:/Users/Ali/logs/restore_2/model.ckpt
Epoch: 0004 cost = 8.907422024
Validation Loss: 8.373998
Model saved in file: C:/Users/Ali/logs/restore_2/model.ckpt
Epoch: 0005 cost = 8.504339825
Validation Loss: 8.133545
Model saved in file: C:/Users/Ali/logs/restore_2/model.ckpt
Epoch: 0006 cost = 8.038612347
Validation Loss: 7.5137353
Model saved in file: C:/Users/Ali/logs/restore_2/model.ckpt
Epoch: 0007 cost = 7.742828803
Validation Loss: 7.7705045
Model saved in file: C:/Users/Ali/logs/restore_2/model.ckpt
Epoch: 0008 cost = 7.517089003
Validation Loss: 10.476646
Model saved in file: C:/Users/Ali/logs/restore_2/model.ckpt
Epoch: 0009 cost = 7.317576204
Validation Loss: 7.921607
Mode

Epoch: 0071 cost = 5.386549206
Validation Loss: 5.355979
Model saved in file: C:/Users/Ali/logs/restore_2/model.ckpt
Epoch: 0072 cost = 5.377431601
Validation Loss: 5.284487
Model saved in file: C:/Users/Ali/logs/restore_2/model.ckpt
Epoch: 0073 cost = 5.373800682
Validation Loss: 5.2851543
Model saved in file: C:/Users/Ali/logs/restore_2/model.ckpt
Epoch: 0074 cost = 5.368290022
Validation Loss: 5.2500587
Model saved in file: C:/Users/Ali/logs/restore_2/model.ckpt
Epoch: 0075 cost = 5.366392793
Validation Loss: 5.2580004
Model saved in file: C:/Users/Ali/logs/restore_2/model.ckpt
Epoch: 0076 cost = 5.365396600
Validation Loss: 5.242038
Model saved in file: C:/Users/Ali/logs/restore_2/model.ckpt
Epoch: 0077 cost = 5.351641964
Validation Loss: 5.2458997
Model saved in file: C:/Users/Ali/logs/restore_2/model.ckpt
Epoch: 0078 cost = 5.349459674
Validation Loss: 5.2859545
Model saved in file: C:/Users/Ali/logs/restore_2/model.ckpt
Epoch: 0079 cost = 5.352064403
Validation Loss: 5.258039
Mo

Epoch: 0141 cost = 5.204268622
Validation Loss: 5.128411
Model saved in file: C:/Users/Ali/logs/restore_2/model.ckpt
Epoch: 0142 cost = 5.199926198
Validation Loss: 5.099066
Model saved in file: C:/Users/Ali/logs/restore_2/model.ckpt
Epoch: 0143 cost = 5.194483899
Validation Loss: 5.11306
Model saved in file: C:/Users/Ali/logs/restore_2/model.ckpt
Epoch: 0144 cost = 5.199389603
Validation Loss: 5.07614
Model saved in file: C:/Users/Ali/logs/restore_2/model.ckpt
Epoch: 0145 cost = 5.198007344
Validation Loss: 5.0766234
Model saved in file: C:/Users/Ali/logs/restore_2/model.ckpt
Epoch: 0146 cost = 5.191464565
Validation Loss: 5.076571
Model saved in file: C:/Users/Ali/logs/restore_2/model.ckpt
Epoch: 0147 cost = 5.181988735
Validation Loss: 5.102293
Model saved in file: C:/Users/Ali/logs/restore_2/model.ckpt
Epoch: 0148 cost = 5.190762404
Validation Loss: 5.0923157
Model saved in file: C:/Users/Ali/logs/restore_2/model.ckpt
Epoch: 0149 cost = 5.194684911
Validation Loss: 5.079742
Model s